<h1 align=center><font size = 5>JT's IBM Data Science Week 3 Assignment:</font></h1>

<h1 align=center><font size = 5>Toronto Neighborhood Clustering</font></h1>

<h3 align=center><font size = 3>This notebook will contain all 3 of the assignment elements. Please therefore note that it will be loaded into Coursera 3 times, once in each submission link field.</font></h3>

<h3 align=center><font size = 3>Each of the 3 sections will be clearly labelled with the prefix SECTION <#>: </font></h3>

<a><img src = "https://upload.wikimedia.org/wikipedia/commons/d/d9/Flag_of_Canada_%28Pantone%29.svg" width = 200> </a>

<h3 align=left><font size = 3>SECTION 1:  Reading and preprocessing data</font></h3>
<h2 align=left><font size = 3>Creating a dataframe based on the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M. </font></h2>

<body>
    
I'll be following the submission guidelines (commenting along the way), and then will upload this notebook into Github and will add the link to the Coursera submission page.</body>

In [1]:
#importing required libraries (adding for entire notebook)
import numpy as np
import pandas as pd 
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

from geopy.geocoders import Nominatim

import folium

#I'm using my Macbook Anaconda Jupyter environment so need to load wget
import wget


Here I am grabbing the postal codes file from wikipedia.  I am using desktop Jupyter (via Anaconda).  

The version of wget in the wild is not the same as the version used by the IBM Skills Network Lab environment hence the slight syntax difference:  

In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
outfile="/Users/jtege/Documents/Coursera IBM Data Sci Spec/9. Applied Data Science Capstone/List_of_postal_codes_of_Canada_M"
wget.download(url, outfile)


'/Users/jtege/Documents/Coursera IBM Data Sci Spec/9. Applied Data Science Capstone/List_of_postal_codes_of_Canada_M'

Rather than write my own parser (which I tried to do...painfully), I'm using the BeautifulSoup library ('bs4').  This will read in the wikipage file and isolate the embedded postal code table, expressed as html.  

Thereafter, I convert the html to a dataframe:

In [3]:
from bs4 import BeautifulSoup

req = requests.get(url)
soup = BeautifulSoup(req.content,'lxml')
tablehtml = soup.find_all('table')[0]

table = pd.read_html(str(tablehtml))
table_df=pd.DataFrame(table[0])
table_df.head()


,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


I assume that I have to get rid of all 'Not assigned' Boroughs...:

In [4]:
toronto = table_df
toronto = toronto.set_index('Borough')
toronto = toronto.drop("Not assigned", axis=0)
toronto = toronto.reset_index(drop=False)

toronto.head()


,Borough,Postal code,Neighborhood
0,North York,M3A,Parkwoods
1,North York,M4A,Victoria Village
2,Downtown Toronto,M5A,Regent Park / Harbourfront
3,North York,M6A,Lawrence Manor / Lawrence Heights
4,Downtown Toronto,M7A,Queen's Park / Ontario Provincial Government


...and I have to replace the slashes with comma delimiters:

In [5]:
toronto['Neighborhood'] = \
 toronto['Neighborhood'].str.replace(" / ", ", ")


In [6]:
#Below set/reset to restore the Postal code col to correct position
toronto = toronto.set_index('Postal code')
toronto = toronto.reset_index(drop=False)
toronto.head()


,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Note I made an assumption that this part of the assignment was informational-only:

"If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough."

My assumption is based on checks I did (see cell below):

In [7]:
#searched for nulls, compared result to same from original wiki table
print('orig table nulls: ', 
      table_df['Neighborhood'].isnull().values.sum())
print('cleaned table nulls: ', 
      toronto['Neighborhood'].isnull().values.sum())


orig table nulls:  77
cleaned table nulls:  0


...lastly for Section 1, am showing dataframe shape

In [8]:
toronto.shape


(103, 3)

<h3 align=left><font size = 3>END OF SECTION 1 -----------</font></h3>




<h3 align=left><font size = 3>SECTION 2:   Adding latitude and longitude data</font></h3>

<body>

    
Here I'll add latitude and longitude data for each postal code in my 'toronto' dataframe such that it matches the frame format illustrated in the submission instructions.
</body>

Note that I was having problems with geocoder and so am using the .csv file:

In [9]:
url2="http://cocl.us/Geospatial_data"
outfile2="/Users/jtege/Documents/Coursera IBM Data Sci Spec/9. Applied Data Science Capstone/Geospatial_data"
wget.download(url2, outfile2)


'/Users/jtege/Documents/Coursera IBM Data Sci Spec/9. Applied Data Science Capstone/Geospatial_data'

In [10]:
#Read in the .csv and set index for faster lookup (later)
latlng_df = pd.read_csv('Geospatial_data')

latlng_df = latlng_df.set_index('Postal Code')

latlng_df.head()


,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


I'll add latitude and longitude columns to my dataframe and will prep the csv postcode info so I can use an index during loading (later):

In [11]:
toronto['Latitude']=None
toronto['Longitude']=None

toronto.head()


,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,None,None
1,M4A,North York,Victoria Village,None,None
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",None,None
3,M6A,North York,"Lawrence Manor, Lawrence Heights",None,None
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",None,None


...and now I'll load up longitude and latitude data for the toronto df post codes by looking them up using the geocoder csv replacement from earlier (now loaded in latlng_df).  So at the end, my toronto df will be complete:

In [12]:
for x, postcode in enumerate(toronto['Postal code']):
    toronto['Latitude'].iloc[x] = \
      latlng_df['Latitude'].loc[toronto['Postal code'].iloc[x]]
    toronto['Longitude'].iloc[x] = \
      latlng_df['Longitude'].loc[toronto['Postal code'].iloc[x]]

toronto.head()


,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895


And finally (for Section 1), I'll show the shape of the dataframe (as per submission guidelines):

In [13]:
toronto.shape


(103, 5)

<h3 align=left><font size = 3>END OF SECTION 2 -----------</font></h3>




<h3 align=left><font size = 3>SECTION 3: Clustering and exploring</font></h3>

<body>

    
In this section, I'll perform K-means clustering on some neighborhoods; this will include visualisations (Folium maps).
</body>

First let's check out a map of Toronto.  For this one I'll use the geocoder library as I only need a single call.  I'll use the location data returned to make my folium map.

In [14]:
address = 'Toronto'

geolocator = Nominatim(user_agent="JTs_toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(
    latitude, longitude))


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [15]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lng, borough, neighborhood in zip(toronto['Latitude'], \
                                    toronto['Longitude'], \
                                    toronto['Borough'], \
                                    toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


As I am low on imagination today, I will follow the suggestion in the submission guidelines and will isolate boroughs referencing 'Toronto'.


In [16]:
toronto_subset = \
toronto[toronto['Borough'].str.contains("Toronto")].reset_index( \
                                                        drop=True )

print('Boroughs with "Toronto" in name are: ', 
      toronto_subset['Borough'].unique())


Boroughs with "Toronto" in name are:  ['Downtown Toronto' 'East Toronto' 'West Toronto' 'Central Toronto']


Ok, I want to know more about "Downtown Toronto" because I'm from a city and "downtown" is always the cool place to be.  (ASSUMPTION: I assume we are allowed to tailor our locations as we choose, as long as there is enough data to do clustering).

I will rebuild the map focussing on that location and will add markers for all the associated neighborhood.  This is all done in the following 2 cells:

In [17]:
address = 'Downtown Toronto'
area_data = \
  toronto[toronto['Borough']==address].reset_index( \
                                                        drop=True )

geolocator = Nominatim(user_agent="JTs_toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of {} are {}, {}.'.format(address, 
                                                latitude, longitude))


The geograpical coordinates of Downtown Toronto are 43.6541737, -79.38081164513409.


In [18]:
map_area = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, borough, neighborhood in zip(area_data['Latitude'], \
                                    area_data['Longitude'], \
                                    area_data['Borough'], \
                                    area_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_area)  
    
map_area


I'll pull some venue information about each of the neighborhoods in this borough using Four Square.

As I do not wish to publish my personal Four Square credentials, I will load up the client id & secret from files on my desktop.

In [19]:
filename1="/Users/jtege/Documents/Coursera IBM Data Sci Spec/9. Applied Data Science Capstone/fsq.clientid"
with open(filename1) as file1:
    CLIENT_ID=file1.read()

filename2="/Users/jtege/Documents/Coursera IBM Data Sci Spec/9. Applied Data Science Capstone/fsq.clientsec"
with open(filename2) as file2:
    CLIENT_SECRET=file2.read()

#print(CLIENT_ID)
#print(CLIENT_SECRET)


Now I will set some parms for my calls later to Four Square, using a radius of 500meters and a venues limit of 100 per call.  Note: I'll use version '20180605' although I know nothing about Four Square versions.

In [20]:
radius=500
LIMIT=100
VERSION = '20180605'


I'll need a function that will loop through and pull the venue data from Foursquare for all my downtown neighborhoods - am setting that up below:

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)            
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)            
        results = requests.get(url).json()["response"]['groups'][0]['items']   
        venues_list.append([(name, lat, lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results] )

    nearby_venues = pd.DataFrame([item for venue_list in 
                                  venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


Now I'll get the Four Square data for all the neighborhoods pulled for Downtown Toronto (each neighborhood will be printed, so I'll get a sense of how many there are in this location)

In [22]:
area_venues = getNearbyVenues(names=area_data['Neighborhood'],
                                   latitudes=area_data['Latitude'],
                                   longitudes=area_data['Longitude']
                                  )


Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


I want to do a few checks to make sure this is enough data to play with.  See below:

In [23]:
print ('A few checks to ensure there is enough data available...')

print ('\ntotal neighborhoods included: ', 
       len(area_venues['Neighborhood'].unique()) )

print ('\narea_data shape: ', area_venues.shape)

print ('\nunique venues: ', 
       len(area_venues['Venue Category'].unique()) )


A few checks to ensure there is enough data available...

total neighborhoods included:  19

area_data shape:  (1225, 7)

unique venues:  204


I'm happy that I have a big enough data set to exercise with so will transpose the venues data (with a 'onehot' technique) so that it is easier to assess later:

In [24]:
area_onehot = pd.get_dummies(area_venues[['Venue Category']], prefix="", prefix_sep="")

area_onehot['Neighborhood'] = area_venues['Neighborhood'] 

#tricking the dataframe to put Neighborhood column back in front
area_onehot.set_index(['Neighborhood'], inplace=True)
area_onehot.reset_index(inplace=True)

#...and now just checking that the table shape is as expected...very wide
print ('original area df shape: ', area_venues.shape)
print ('onehot area df shape: ', area_onehot.shape)


original area df shape:  (1225, 7)
onehot area df shape:  (1225, 204)


I will now have a look at the Neighborhood data by a simple grouping of the dataframe entries by Neighborhood.  Afterward, I'll print the top few most prevalent venues for each given Neighborhood.  See next 2 cells:

In [25]:
area_grouped = area_onehot.groupby('Neighborhood').mean().reset_index()
area_grouped.head()


,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.017544,0.0,0.0,0.0,0.0,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
2,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.015873,0.0,0.0,0.0,0.0,0.015873
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
4,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,0.0,0.0,...,0.012821,0.012821,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.025641


To make a neater version of this, I'll mimic what we did for the New York analysis and create a tidy tiny dataframe:

In [26]:
top_venues_lim = 3


In [27]:
def return_most_common_venues(row, top_venues_lim):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:top_venues_lim]


In [28]:
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']

for ind in np.arange(top_venues_lim):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, 
                                                       indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

area_venues_sorted = pd.DataFrame(columns=columns)
area_venues_sorted['Neighborhood'] = area_grouped['Neighborhood']

for ind in np.arange(area_grouped.shape[0]):
    area_venues_sorted.iloc[ind, 1:] = \
    return_most_common_venues(area_grouped.iloc[ind, :], top_venues_lim)

area_venues_sorted


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal
2,Central Bay Street,Coffee Shop,Italian Restaurant,Café
3,Christie,Grocery Store,Café,Park
4,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant
5,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café
6,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel
7,"Garden District, Ryerson",Clothing Store,Coffee Shop,Café
8,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Hotel
9,"Kensington Market, Chinatown, Grange Park",Café,Coffee Shop,Mexican Restaurant


Ok, this is kinda cool.  I've never been to Toronto and can see from this that if I had kids I would probably want to live in "Christie" or "Rosedale" given the relative number of parks and playgrounds.  But if I were a hipster, I might want to live near the "Kensington Market...", or  "Regent Park, Harbourfront" or similar.  I'll make a note...

Now finally I'll try clustering using k-means.  As my dataset is not massive, I won't go for too many clusters...no problem as I believe this is just to show that I know how to use the function. (Assumption Alert!)

In [29]:
kclusters = 5

area_grouped_clustering = area_grouped.drop('Neighborhood', axis=1)

kmeans = KMeans(n_clusters=kclusters, \
                random_state=0).fit(area_grouped_clustering)

kmeans.labels_[0:10] 

array([1, 0, 1, 2, 1, 1, 1, 1, 1, 4], dtype=int32)

In [30]:
area_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

area_merged = area_data

area_merged = area_merged.join(area_venues_sorted.set_index \
                               ('Neighborhood'), \
                               on='Neighborhood')

area_merged


,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606,1,Coffee Shop,Pub,Park
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895,1,Coffee Shop,Sushi Restaurant,Diner
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789,1,Clothing Store,Coffee Shop,Café
3,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754,1,Café,Coffee Shop,Gastropub
4,M5E,Downtown Toronto,Berczy Park,43.6448,-79.3733,1,Coffee Shop,Cocktail Bar,Beer Bar
5,M5G,Downtown Toronto,Central Bay Street,43.658,-79.3874,1,Coffee Shop,Italian Restaurant,Café
6,M6G,Downtown Toronto,Christie,43.6695,-79.4226,2,Grocery Store,Café,Park
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.6506,-79.3846,1,Coffee Shop,Café,Restaurant
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.6408,-79.3818,1,Coffee Shop,Aquarium,Hotel
9,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.6472,-79.3816,1,Coffee Shop,Hotel,Café


And finally, I'll show these clusters (as markers) on a Folium map:

In [31]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(area_merged['Latitude'], area_merged['Longitude'], area_merged['Neighborhood'], area_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


Given what kmeans on neighborhoods created, I'd say: 
- Cluster0 is the airport
- Cluster1 is where coffee shops and cafes are most prevalent
- Cluster2 is off the beaten path, possibly more family/residential
- Cluster3 is more outdoor and parkland
- Cluster4 is for the hipsters

<h3 align=left><font size = 3>END OF SECTION 3 -----------</font></h3>




<h1 align=left><font size = 3>END OF ASSIGNMENT</font></h1>